<a href="https://colab.research.google.com/github/osullik/ic23050/blob/main/DataDictionary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
from google.colab import drive
import pandas as pd
import json

In [6]:
DD = pd.ExcelFile("https://github.com/osullik/ic23050/blob/main/L4%20Washington%20Fatal%20Crash%20Files%20-%20Washington%20Traffic%20Safety%20Commission/DataDictionary_Washington%20Fatal%20Crash%20Files.xlsx?raw=true")

In [19]:
for sheet in DD.sheet_names:
  print(sheet)
  


ReadMe
CRASH Data Files
PERSON Data Files
Tables1
Tables2
Tables3
Tables4
Tables5
Tables6
Tables7
Tables8
Tables9


In [24]:
ReadMe = pd.read_excel(DD, 'ReadMe')
CrashDataFiles = pd.read_excel(DD, 'CRASH Data Files')
PersonDataFiles = pd.read_excel(DD, 'PERSON Data Files')
Tables1 = pd.read_excel(DD, 'Tables1')
Tables2 = pd.read_excel(DD, 'Tables2')
Tables3 = pd.read_excel(DD, 'Tables3')
Tables4 = pd.read_excel(DD, 'Tables4')
Tables5 = pd.read_excel(DD, 'Tables5')
Tables6 = pd.read_excel(DD, 'Tables6')
Tables7 = pd.read_excel(DD, 'Tables7')
Tables8 = pd.read_excel(DD, 'Tables8')
Tables9 = pd.read_excel(DD, 'Tables9')

In [89]:
print(CrashDataFiles.columns)

#for column in CrashDataFiles.columns:
#  print("Column is:", column)

numerals = [1,2,3,4,5,6,7,8,9,0]
for idx, series in CrashDataFiles.iterrows():
  var = series['VARIABLE']
  desc =series['DESCRIPTION/COMMENT']
  
  if type(desc) == str and desc.find("=") == True:
    #print(desc)
    lineSplit = (desc.split("\n"))
    for line in lineSplit:
      kvPair = line.split('=')
      if len(kvPair) >2:
      #$print(kvPair)
        for item in kvPair:
          if any(item) in numerals:
            print(item)


Index(['Unnamed: 0', 'VARIABLE', 'TYPE', 'LENGTH', 'FORMAT', 'LABEL',
       'DESCRIPTION/COMMENT', 'Notes / SAS Code'],
      dtype='object')
7
Oil                                                                                                                                                                                                                                                     8
Other
0
No   1
Yes
5
Front-to-Side, Right Angle (e.g., 'broadside')                                                                                                                                           6
Angle
8
Sideswipe, Opposite Direction                                                                                                                                                                                                                                                     9
Rear-to-Side
0
No            1
Yes     
8
Not reported            9
Unknown
1
No Intersection                    